In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Pheochromocytoma_and_Paraganglioma"
cohort = "GSE19987"

# Input paths
in_trait_dir = "../../input/GEO/Pheochromocytoma_and_Paraganglioma"
in_cohort_dir = "../../input/GEO/Pheochromocytoma_and_Paraganglioma/GSE19987"

# Output paths
out_data_file = "../../output/preprocess/Pheochromocytoma_and_Paraganglioma/GSE19987.csv"
out_gene_data_file = "../../output/preprocess/Pheochromocytoma_and_Paraganglioma/gene_data/GSE19987.csv"
out_clinical_data_file = "../../output/preprocess/Pheochromocytoma_and_Paraganglioma/clinical_data/GSE19987.csv"
json_path = "../../output/preprocess/Pheochromocytoma_and_Paraganglioma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Germline Mutations in TMEM127 Confer Susceptibility to Pheochromocytoma"
!Series_summary	"Pheochromocytomas, catecholamine-secreting tumors of neural crest origin, are frequently hereditary. However, the molecular basis of the majority of these tumors is unknown. We identified the transmembrane-encoding gene TMEM127 on chromosome 2q11 as a new pheochromocytoma susceptibility gene. In a cohort of 103 samples, we detected truncating germline TMEM127 mutations in approximately 30% of familial tumors and about 3% of sporadic-appearing pheochromocytomas without a known genetic cause. The wild-type allele was consistently deleted in tumor DNA, suggesting a classic mechanism of tumor suppressor gene inactivation. Pheochromocytomas with mutations in TMEM127 are transcriptionally related to tumors bearing NF1 mutations and, similarly, show hyperphosphorylation of mammalian target of rapamycin (mTOR) effector proteins. Accordingly, in vitro gain-of-function

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
is_gene_available = True  # Based on the Series_title and Series_summary, this dataset contains gene expression data from pheochromocytomas

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
trait_row = 1  # Based on 'genetic class' that indicates whether samples have TMEM127 mutations or other genetic variants
age_row = None  # Age information not available in the sample characteristics
gender_row = None  # Gender information not available in the sample characteristics

# 2.2 Data Type Conversion
def convert_trait(value):
    if not isinstance(value, str):
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # The trait is "Pheochromocytoma_and_Paraganglioma" with genetic class as subtype
    # Convert genetic classes to binary (FP_TM = TMEM127 mutation vs others)
    if value == "FP_TM":  # FP_TM appears to be the TMEM127 mutation group based on the study description
        return 1  # Positive for TMEM127 mutation
    elif value in ["MEN2A", "B_SDHB", "SPOR", "VHL", "NF1", "KIF", "D_SDHD", "F_other"]:
        return 0  # Other genetic classes (no TMEM127 mutation)
    else:
        return None

# Define convert_age and convert_gender functions even though they won't be used
def convert_age(value):
    return None  # Age data not available

def convert_gender(value):
    return None  # Gender data not available

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. We're skipping detailed clinical feature extraction at this step
# because it appears we only have metadata about available variables,
# not the actual clinical data for individual samples.
# We've already recorded that trait data is theoretically available.
print(f"Gene expression data available: {is_gene_available}")
print(f"Trait data available: {is_trait_available}")
print(f"Metadata saved to {json_path}")


Gene expression data available: True
Trait data available: True
Metadata saved to ../../output/preprocess/Pheochromocytoma_and_Paraganglioma/cohort_info.json


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1598_g_at', '160020_at', '1729_at', '1773_at', '177_at',
       '179_at', '1861_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers look like Affymetrix probe IDs (e.g., "1007_s_at"), not human gene symbols.
# Affymetrix probe IDs typically end with "_at" and need to be mapped to gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Observe and identify the appropriate columns
# From the preview we can see:
# - 'ID' column contains the probe IDs (like '1007_s_at') which match the gene expression data index
# - 'Gene Symbol' column contains the human gene symbols we need to map to

# 2. Get the gene mapping dataframe using the get_gene_mapping function from the library
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')

print("Gene mapping preview:")
print(preview_df(gene_mapping))

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(gene_data, gene_mapping)

print("\nGene-level expression data preview:")
print(gene_data.shape)
print(list(gene_data.index[:20]))  # Display first 20 gene symbols


Gene mapping preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'Gene': ['DDR1 /// MIR4640', 'RFC2', 'HSPA6', 'PAX8', 'GUCA1A']}



Gene-level expression data preview:
(13830, 50)
['A1CF', 'A2M', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC', 'AAGAB', 'AAK1', 'AAMDC', 'AAMP', 'AANAT', 'AAR2', 'AARS', 'AARSD1', 'AASDHPPT', 'AASS', 'AATF', 'AATK', 'ABAT']


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. First, properly extract and process clinical data
# Since we skipped this step earlier, let's do it now
clinical_features = geo_select_clinical_features(
    clinical_df=clinical_data, 
    trait=trait, 
    trait_row=trait_row, 
    convert_trait=convert_trait
)

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Shape after handling missing values: {linked_data.shape}")

# 4. Determine whether the trait and demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note=f"Dataset contains gene expression data for {trait} samples with TMEM127 mutation classification."
)

# 6. Save the data if it's usable
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data quality check failed. The dataset is not suitable for association studies.")

Normalized gene data saved to ../../output/preprocess/Pheochromocytoma_and_Paraganglioma/gene_data/GSE19987.csv
Clinical data saved to ../../output/preprocess/Pheochromocytoma_and_Paraganglioma/clinical_data/GSE19987.csv
Linked data shape: (50, 13543)


Shape after handling missing values: (49, 13543)
For the feature 'Pheochromocytoma_and_Paraganglioma', the least common label is '1.0' with 1 occurrences. This represents 2.04% of the dataset.
The distribution of the feature 'Pheochromocytoma_and_Paraganglioma' in this dataset is severely biased.

Data quality check failed. The dataset is not suitable for association studies.
